### URL da API utilizada: https://viacep.com.br/ws/---- CEP -----/json

### Realização de um ELT

### Extract
##### Extrair os CEPs informados no .csv 'CEPs.csv' para obter os dados faltantes de localização dos CEPs do Brasil e gera um novo arquivo para retorno.

In [33]:
import pandas as pd
import requests
import json
api_correios = 'https://viacep.com.br/ws/'

In [34]:
df = pd.read_csv('Dados/CEPs.csv')

In [35]:
# Função para padronizar os CEPs
def padronizar_cep(cep):
    cep = cep.replace('-', '')  # Remover traços se presentes
    return f'{cep[:5]}{cep[5:]}' if len(cep) == 8 and cep.isdigit() else 'CEP Invalido'

# Aplicar a função ao DataFrame
df['CEP_padronizado'] = df['CEP'].apply(padronizar_cep)

In [36]:

df = df[df['CEP_padronizado'] != 'CEP Invalido'] # Remove os CEPs que não são "válidos"

In [37]:
lista_ceps = df['CEP_padronizado'].tolist()

In [38]:
# Função para pegar o usuário dentro da API
def get_cep(id):
    response = requests.get(f'{api_correios}{id}/json')# Puxa os dados do usuário contendo o cep solicitado
    return response.json() if response.status_code == 200 else None # Tratamento de erro caso não seja encontrado o cep solicitado


In [39]:
ceps = [cep for id in lista_ceps if (cep := get_cep(id)) is not None ]# Atribui o usuário na variável apenas se ele não for nulo.


### Transform
##### Transforma os dados retornados da API em um DataFrame para manipulação

In [40]:
df_ceps = pd.DataFrame(ceps)
display(df_ceps)

,cep,logradouro,complemento,bairro,localidade,uf,ibge,gia,ddd,siafi,erro
0,22790-022,Rua Raul da Cunha Ribeiro,,Recreio dos Bandeirantes,Rio de Janeiro,RJ,3304557,,21,6001,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,20040-906,Avenida Rio Branco,109,Centro,Rio de Janeiro,RJ,3304557,,21,6001,NaN
4,20211-972,Rua Afonso Cavalcanti,22 4 Andar,Cidade Nova,Rio de Janeiro,RJ,3304557,,21,6001,NaN
5,02040-070,Rua Barra de São João,,Jardim São Paulo(Zona Norte),São Paulo,SP,3550308,1004,11,7107,NaN
6,04843-475,Viela São Paulo,,Parque São José,São Paulo,SP,3550308,1004,11,7107,NaN
7,08430-028,Travessa Baía de Emboraí,(Cj Hab Guaianazes),Jardim Santa Terezinha (Zona Leste),São Paulo,SP,3550308,1004,11,7107,NaN
8,22780-085,Estrada dos Bandeirantes,de 6535 a 7993 - lado ímpar,Jacarepaguá,Rio de Janeiro,RJ,3304557,,21,6001,NaN


In [41]:
df_ceps = df_ceps.drop(labels="erro", axis=1) # Remove a tabela "erro"
df_ceps


,cep,logradouro,complemento,bairro,localidade,uf,ibge,gia,ddd,siafi
0,22790-022,Rua Raul da Cunha Ribeiro,,Recreio dos Bandeirantes,Rio de Janeiro,RJ,3304557,,21,6001
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20040-906,Avenida Rio Branco,109,Centro,Rio de Janeiro,RJ,3304557,,21,6001
4,20211-972,Rua Afonso Cavalcanti,22 4 Andar,Cidade Nova,Rio de Janeiro,RJ,3304557,,21,6001
5,02040-070,Rua Barra de São João,,Jardim São Paulo(Zona Norte),São Paulo,SP,3550308,1004,11,7107
6,04843-475,Viela São Paulo,,Parque São José,São Paulo,SP,3550308,1004,11,7107
7,08430-028,Travessa Baía de Emboraí,(Cj Hab Guaianazes),Jardim Santa Terezinha (Zona Leste),São Paulo,SP,3550308,1004,11,7107
8,22780-085,Estrada dos Bandeirantes,de 6535 a 7993 - lado ímpar,Jacarepaguá,Rio de Janeiro,RJ,3304557,,21,6001


In [42]:
nan_df = df_ceps[df_ceps.isna().any(axis=1)] # Localiza todas as linhas com valores NaN
nan_df

,cep,logradouro,complemento,bairro,localidade,uf,ibge,gia,ddd,siafi
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df_ceps = df_ceps.dropna(how='all') # Remove todas as linhas com valores NaN
df_ceps

,cep,logradouro,complemento,bairro,localidade,uf,ibge,gia,ddd,siafi
0,22790-022,Rua Raul da Cunha Ribeiro,,Recreio dos Bandeirantes,Rio de Janeiro,RJ,3304557,,21,6001
3,20040-906,Avenida Rio Branco,109,Centro,Rio de Janeiro,RJ,3304557,,21,6001
4,20211-972,Rua Afonso Cavalcanti,22 4 Andar,Cidade Nova,Rio de Janeiro,RJ,3304557,,21,6001
5,02040-070,Rua Barra de São João,,Jardim São Paulo(Zona Norte),São Paulo,SP,3550308,1004,11,7107
6,04843-475,Viela São Paulo,,Parque São José,São Paulo,SP,3550308,1004,11,7107
7,08430-028,Travessa Baía de Emboraí,(Cj Hab Guaianazes),Jardim Santa Terezinha (Zona Leste),São Paulo,SP,3550308,1004,11,7107
8,22780-085,Estrada dos Bandeirantes,de 6535 a 7993 - lado ímpar,Jacarepaguá,Rio de Janeiro,RJ,3304557,,21,6001


In [44]:
df_ceps_final = df_ceps[['cep', 'logradouro', 'bairro', 'localidade', 'uf']] # Seleciona apenas as colunas necessárias para retorno.
df_ceps_final

,cep,logradouro,bairro,localidade,uf
0,22790-022,Rua Raul da Cunha Ribeiro,Recreio dos Bandeirantes,Rio de Janeiro,RJ
3,20040-906,Avenida Rio Branco,Centro,Rio de Janeiro,RJ
4,20211-972,Rua Afonso Cavalcanti,Cidade Nova,Rio de Janeiro,RJ
5,02040-070,Rua Barra de São João,Jardim São Paulo(Zona Norte),São Paulo,SP
6,04843-475,Viela São Paulo,Parque São José,São Paulo,SP
7,08430-028,Travessa Baía de Emboraí,Jardim Santa Terezinha (Zona Leste),São Paulo,SP
8,22780-085,Estrada dos Bandeirantes,Jacarepaguá,Rio de Janeiro,RJ


### Load
##### Atualizar criando um novo arquivo CVS com os dados necessários para retorno.

In [45]:
df_ceps_final.to_csv('./Dados/CEPs_Final.csv', index=False) # Cria um novo arquivo .csv para adicionar a informação encontrada. 